<a href="https://colab.research.google.com/github/luisgustavoreimberg/CurriculOn/blob/master/Desafio_Imersao_Alura_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Realizando configurações iniciais

Instalação do SDK do Google

In [3]:
!pip install -q -U google-generativeai

Importando dependências

In [15]:
import google.generativeai as genai
from google.colab import userdata
import os
import time
from google.api_core.exceptions import TooManyRequests
import logging

Configurando o modelo

# 2 - Configurando o modelo

Inserindo API Key

In [6]:
genai.configure(api_key = userdata.get('apiKey'))

Configurando padrâmetros do modelo

In [7]:
#CONFIGURANDO PARÂMETROS DO MODELO
generation_config = genai.GenerationConfig(
  candidate_count = 1,
)
safety_settings = {
    "HARASSMENT": "BLOCK_ONLY_HIGH",
    "HATE": "BLOCK_ONLY_HIGH",
    "SEXUAL": "BLOCK_ONLY_HIGH",
    "DANGEROUS": "BLOCK_ONLY_HIGH"
}

#CONFIGURANDO INSTRUÇÕES DO SISTEMA PARA DIRECIONAR O CONTEXTO, PERSONA E REGRAS
system_instruction = [
    "Você é um atendente de uma central de emergência via chat",
    "Você deve realizar o atendimento inicial e classificar de forma discreta se é uma emergência ou não(um trote)",
    "Você deve realizar no mínimo 3 pergunta para o atendimento",
    "Você deve buscar informações sobre o ocorrido, identificando no mínimo o local do ocorrido e quantidade de envolvidos"
    "O atendimento deve ser guiado por mensagens ou questionamentos simples e únicos"
]

Gerando o modelo

In [8]:
model = genai.GenerativeModel(
    model_name = "gemini-1.5-pro-latest",
    generation_config = generation_config,
    system_instruction = system_instruction,
    safety_settings = safety_settings
)

# 3 - Executando o Chat

Incluindo métodos úteis para reaproveitamento de trechos de código envolvendo mensagems de console

In [12]:
def console_clear():
    os.system('cls') #COMANDO PARA LIMPAR CONSOLE WINDOWS

def console_message(message, role = 'SYSTEM', must_clear = False, with_header = False, with_footer = False):
    if with_header:
        print("----------------------------------------------")
        print("-- S.O.S -------------------------------------")
    if(type(message) == str):
        message = [message]
    for msg in message:
        print(f"{role.upper()}: {msg}")
    if with_footer:
        print("------------------------versão de testes/estudo")
        print("----------------------------------------------")

def console_question(message, role = 'SYSTEM', must_get_answer = True):
    question = f"{role.upper()}: {message}\nUSER: "
    answer = input(question)
    if must_get_answer:
        while not answer:
            console_clear()
            console_message("ATENÇÃO, RESPONSA A PERGUNTA CONFORME SOLICITADO")
            answer = input(question)
    return answer

def console_chat_history(chat_history):
    time.sleep(5)
    console_message("INÍCIO HISTÓRICO", must_clear= True, with_header = True)

    if(len(chat.history) > 0):
      for content in chat_history:
        console_message(content.parts[0].text, content.role)

    console_message("FIM", with_footer = True)

Incluindo variaveis utilizadas no chat

In [16]:
logging.getLogger("tornado.access").setLevel(logging.CRITICAL) #removendo logging do console

user_escape_mensage = "SAIR"
bot_escape_message = "#FIM#"

mensagens_iniciais_usuario = [
    "ATENÇÃO, UTILIZE O SERVIÇO COM RESPONSABILIDADE",
    "RESPONDA AS PERGUNTAS CORRETAMENTE PARA DEFINIRMOS O ATENDIMENTO EMERGENCIAL",
    f"PARA FINALIZAR SEU ATENDIMENTO DIGITE \"{user_escape_mensage}\""]

initial_bot_prompt = f"Quando você se decidir sobre o atendimento, envie uma mensagem no seguinte padrão: Quando é um trote ou não emergência: 'Estarei finalizando sua ligação{bot_escape_message}' e Quando é uma emergência: 'Estamos enviando uma unidade de atendimento o mais rápido possível{bot_escape_message}'"

Gerando estrutura dos chats

In [17]:
try:
  #INICIANDO O CHAT
  console_message(message = mensagens_iniciais_usuario, with_header = True)

  #INICIANDO O CHAT E ENVIANDO PROMPT INICIAL
  chat = model.start_chat(history=[])
  ai_response = chat.send_message(initial_bot_prompt).text

  #FLUXO DO CHAT
  while not bot_escape_message in ai_response:
    time.sleep(5)#DIMINUINDO FREQUÊNCIA DAS REQUISIÇÕES PARA EVITAR ERRO DE TOO MANY REQUESTS
    prompt = console_question(ai_response, "BOT")
    if prompt == user_escape_mensage:
      console_message("ATENDIMENTO FINALIZADO PELO USUÁRIO")
      break
    ai_response = chat.send_message(prompt).text

  console_message(ai_response.replace(bot_escape_message, ""), "BOT")

except TooManyRequests:
  #CAPTURANDO MENSAGEM DE MUITAS REQUISIÇÕES(RATE LIMIT)
  console_message("NOSSOS SISTEMAS ESTÃO MUITO OCULPADOS. SEU ATENDIMENTO ESTÁ SENDO TRANSFERIDO PARA UM AGENTE!", must_clear = True, with_header = True)

except Exception as e:
  #CAPTURANDO ERROS
  console_message(f"OCORREU UM ERRO DURANTE A REQUISIÇÃO. SEU ATENDIMENTO ESTÁ SENDO TRANSFERIDO AGENTE!", must_clear = True, with_header = True)

finally:
  #FINALIZANDO E MOSTRANDO O CONSOLE
  console_message("ATENDIMENTO FINALIZADO", with_footer = True)
  console_chat_history(chat.history)

----------------------------------------------
-- S.O.S -------------------------------------
SYSTEM: ATENÇÃO, UTILIZE O SERVIÇO COM RESPONSABILIDADE
SYSTEM: RESPONDA AS PERGUNTAS CORRETAMENTE PARA DEFINIRMOS O ATENDIMENTO EMERGENCIAL
SYSTEM: PARA FINALIZAR SEU ATENDIMENTO DIGITE "SAIR"
BOT: Olá! Qual o endereço da emergência? 

USER: Rua xyz
BOT: Ok, Rua xyz. Pode me dizer qual o número da casa, por favor? E o que está acontecendo? 

USER: altura do numero 33, acidente de transito
BOT: Rua xyz, número 33. Entendi. Há quantas pessoas envolvidas no acidente? 

USER: Apenas uma pessoa, um dos motoristas
BOT: Estamos enviando uma unidade de atendimento o mais rápido possível 

SYSTEM: ATENDIMENTO FINALIZADO
------------------------versão de testes/estudo
----------------------------------------------
----------------------------------------------
-- S.O.S -------------------------------------
SYSTEM: INÍCIO HISTÓRICO
USER: Quando você se decidir sobre o atendimento, envie uma mensagem no 